<a href="https://colab.research.google.com/github/Devansh-react/PyTorch_fundamentals/blob/main/Ch_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import kagglehub
path = kagglehub.dataset_download("zalando-research/fashionmnist")

In [ ]:
df = pd.read_csv(path + '/fashion-mnist_train.csv')
df.shape

In [ ]:
data_test= pd.read_csv(path + '/fashion-mnist_test.csv')
data_test.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using the:{device}")

In [ ]:
# Create a 4x4 grid of images
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
fig.suptitle("First 16 Images", fontsize=16)

# Plot the first 16 images from the dataset
for i, ax in enumerate(axes.flat):
    img = df.iloc[i, 1:].values.reshape(28, 28)  # Reshape to 28x28
    ax.imshow(img)  # Display in grayscale
    ax.axis('off')  # Remove axis for a cleaner look
    ax.set_title(f"Label: {df.iloc[i, 0]}")  # Show the label

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit the title
plt.show()


In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# Fill NaN values with 0 before splitting and normalization
X = np.nan_to_num(X, nan=0.0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train/255.0
X_test = X_test/255.0
X_train,X_test

In [ ]:
# custom class dataset

class customDataset(Dataset):

  def __init__(self,feature,lables):
    self.feature = torch.tensor(feature,dtype=torch.float32)
    self.lables = torch.tensor(lables,dtype=torch.long)

  def __len__(self):
    return len(self.feature)

  def __getitem__(self, index):

    return self.feature[index],self.lables[index]

In [ ]:
train_Dataset = customDataset(X_train,y_train)
test_Dataset = customDataset(X_test,y_test)

In [ ]:
train_dataloder = DataLoader(train_Dataset,batch_size=128,shuffle=True,pin_memory=True)
#  pin memory=True for faster training
test_dataloader = DataLoader(test_Dataset,batch_size=128,shuffle=False, pin_memory=True)

In [ ]:
# NN

class NNmodule(nn.Module):
  def __init__(self,num_feature):
    super().__init__()
    self.Model = nn.Sequential(
        nn.Linear(num_feature,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(), # Added back the ReLU activation
        nn.Linear(64,10)
    )

  def forward(self,X):

    return self.Model(X)

In [ ]:
epochs=70
learning_rate = 0.01

In [ ]:
model = NNmodule(X_train.shape[1])
model = model.to(device)

loss_fun = nn.CrossEntropyLoss()

optimiser = optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_feature,batch_lable in train_dataloder:

    # move to GPU
    batch_feature = batch_feature.to(device)
    batch_lable = batch_lable.to(device)

    output = model(batch_feature)

    loss = loss_fun(output,batch_lable)

    optimiser.zero_grad()
    loss.backward()

    optimiser.step()
    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_dataloder)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

In [ ]:
model.eval()
total = 0
correct = 0

for batch_feature , batch_lable in test_dataloader:
  # move to  GPU
  batch_feature = batch_feature.to(device)
  batch_lable = batch_lable.to(device)
  output  = model(batch_feature)
  _,predicted = torch.max(output,1)
  #  no of rows every batch
  total += batch_feature.shape[0]
  #  no of max value from each tensor that have value same as lables means predicted == actual
  correct += (predicted == batch_lable).sum().item()

print(correct/total)
